In [ ]:
# Author: Jin Zhang

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', None)
plt.rcParams.update({'font.size': 22})



In [ ]:
projects = pd.read_csv('./data/opendata_projects000.gz', escapechar='\\', \
                           names=['_projectid', '_teacher_acctid', '_schoolid', 'school_ncesid', 'school_latitude', \
                                  'school_longitude', 'school_city', 'school_state', 'school_zip', 'school_metro', \
                                  'school_district', 'school_county', 'school_charter', 'school_magnet', \
                                  'school_year_round', 'school_nlns', 'school_kipp', 'school_charter_ready_promise', \
                                  'teacher_prefix', 'teacher_teach_for_america', 'teacher_ny_teaching_fellow', \
                                  'primary_focus_subject', 'primary_focus_area' ,'secondary_focus_subject', \
                                  'secondary_focus_area', 'resource_type', 'poverty_level', 'grade_level', \
                                  'vendor_shipping_charges', 'sales_tax', 'payment_processing_charges', \
                                  'fulfillment_labor_materials', 'total_price_excluding_optional_support', \
                                  'total_price_including_optional_support', 'students_reached', \
                                  'total_donations', 'num_donors', 'eligible_double_your_impact_match', \
                                  'eligible_almost_home_match', 'funding_status', 'date_posted', 'date_completed', \
                                  'date_thank_you_packet_mailed', 'date_expiration'],\
                       parse_dates= ['date_posted', 'date_completed','date_thank_you_packet_mailed', 'date_expiration']
                      )


In [ ]:
resources = pd.read_csv('./data/opendata_resources000.gz', escapechar='\\', \
                        names=['_resourceid', '_projectid', 'vendorid', 'vendor_name', 'item_name',\
                               'item_number', 'item_unit_price', 'item_quantity'])

In [ ]:
essays = pd.read_csv('./data/opendata_essays000.gz', escapechar='\\', \
                         names=['_projectid', '_teacher_acctid', 'title', 'short_description', 'need_statement', \
                                'essay', 'thankyou_note', 'impact_letter'])

In [ ]:
resources['item_total_expense'] = resources.item_unit_price * resources.item_quantity

In [ ]:
df_exp_item = resources.loc[resources.groupby('_projectid')['item_total_expense'].transform('max') \
                            == resources['item_total_expense']\
                            ,:].sort_values(['_projectid','item_number'])\
                        .groupby('_projectid',as_index=False).first().drop('_resourceid',axis=1)


In [ ]:
df_projects = projects[projects.date_posted>='2015-05-01']

df_first_project = df_projects.loc[df_projects.groupby('_teacher_acctid')['date_posted'].transform('min')\
                ==df_projects['date_posted'],:].reset_index(drop=True)                                                                                 

In [ ]:
#df_first_project['project_count'] = df_first_project.groupby('_teacher_acctid')['_projectid'].transform('count')

In [ ]:
df_first_project['posted_month'] = df_first_project.date_posted.apply(lambda x: x.month)
df_first_project['posted_year'] = df_first_project.date_posted.apply(lambda x: x.year)
df_first_project['funding_success'] = (df_first_project.funding_status == 'completed').astype(int)

In [ ]:
df = df_first_project.merge(essays.drop(['thankyou_note','impact_letter'],axis=1), \
                       how = 'left', on = ['_projectid','_teacher_acctid'])\
                .merge(df_exp_item, how = 'left', on = ['_projectid'])

In [ ]:
df = df[~pd.isnull(df.primary_focus_subject)]
df = df[~pd.isnull(df.title)]

In [ ]:
df['item_expense'] = df.total_price_excluding_optional_support - df.sales_tax\
                      - df.vendor_shipping_charges - df.payment_processing_charges

In [ ]:
# plot on distributions

In [ ]:
g = df_first_project.groupby(['posted_year','posted_month'])[['_projectid']].count()

ax = g.plot(figsize=(20,10), rot=90, grid = True, title= 'total_count')
ax.set_xticks(range(0, g.size,1));
ax.set_xticklabels(["%d-%02d" % item for index, item in enumerate(g.index.tolist()) if index % 1 ==0], rotation=90);

In [ ]:
plt.figure()
g = df_first_project.groupby(['posted_year','posted_month'])[['funding_success']].mean()

ax = g.plot(figsize=(20,10), rot=90, grid = True, title= 'probability of success')
ax.set_xticks(range(0, g.size,1));
ax.set_xticklabels(["%d-%02d" % item for index, item in enumerate(g.index.tolist()) if index % 1 ==0], \
                   rotation=90, fontsize=15);
plt.savefig('prob_success.png',fmt='png', dpi=300, bbox_inches='tight')

In [ ]:
df_use = df[df.date_posted<'2016-05-01'].reset_index(drop=True)
df_outsample = df[df.date_posted>='2016-05-01'].reset_index(drop=True)

In [ ]:
df_use.to_csv('./input/df_use.csv',header=True, index=False) 
df_outsample.to_csv('./input/df_outsample.csv',header=True, index=False) 
